In [8]:
import sys
sys.path.append(r'/data/jykang/NeMo')

import pytorch_lightning as pl
from omegaconf import OmegaConf

from nemo.collections.asr.models.ctc_bpe_models_AddLayer import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

import torch

import yaml
config_path = '/data/jykang/NeMo/examples/asr/asr_ctc/asr_ctc_distil/conf/conformer_ctc_bpe_Fitnet_AddLayer.yaml'
with open (config_path, 'r') as f:
    config = OmegaConf.load(f)
    
def train(cfg):
   
    asr_model = EncDecCTCModelBPE(cfg = cfg['model'], trainer =None)
    
    te_tmp_bias = asr_model.state_dict()['decoder.decoder_layers.0.bias']
    te_tmp_weight = asr_model.state_dict()['decoder.decoder_layers.0.weight']
    a = torch.rand(129)
    a = te_tmp_bias.clone()
    print(a.shape, a)
    print(te_tmp_bias.shape, te_tmp_bias)
    # Initialize the weights of the model from another model, if provided via config
    asr_model.maybe_init_from_pretrained_checkpoint(cfg)
    
    # # teacher decoder load
    # te_dec_bias = torch.load('/data/jykang/NeMo/data/decoder/te_dec_bias.pt')
    # te_dec_weight = torch.load('/data/jykang/NeMo/data/decoder/te_dec_weight.pt')
    # with torch.no_grad():
    #     asr_model.state_dict()['decoder.decoder_layers.0.bias'].copy_(te_dec_bias)
    #     asr_model.state_dict()['decoder.decoder_layers.0.weight'].copy_(te_dec_weight)
    
    with torch.no_grad():
        asr_model.state_dict()['decoder.decoder_layers.0.bias'].copy_(a)
        asr_model.state_dict()['decoder.decoder_layers.0.weight'].copy_(te_tmp_weight)
    return a, asr_model.state_dict()['decoder.decoder_layers.0.bias'], te_tmp_bias
    

a, b, c = train(config)


[NeMo I 2024-05-03 21:57:19 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens
[NeMo I 2024-05-03 21:57:19 ctc_bpe_models_AddLayer:63] 
    Replacing placeholder number of classes (-1) with actual number of classes - 128
[NeMo I 2024-05-03 21:57:39 collections:196] Dataset loaded with 278627 files totalling 948.67 hours
[NeMo I 2024-05-03 21:57:39 collections:197] 2614 files were filtered totalling 12.38 hours
[NeMo I 2024-05-03 21:57:40 collections:196] Dataset loaded with 2703 files totalling 5.39 hours
[NeMo I 2024-05-03 21:57:40 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-05-03 21:57:40 collections:196] Dataset loaded with 2620 files totalling 5.40 hours
[NeMo I 2024-05-03 21:57:40 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-05-03 21:57:40 features:289] PADDING: 0
torch.Size([129]) tensor([-0.0063, -0.0259,  0.0100,  0.0430,  0.0389,  0.0362,  0.0263, -0.0033,
         0.0153, -0.0043, -0.0227,  0.0037,

[NeMo W 2024-05-03 21:57:40 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/jykang/database/train_clean_100.json,/home/jykang/database/train_clean_360.json,/home/jykang/database/train_other_500.json
    sample_rate: 16000
    batch_size: 24
    shuffle: false
    num_workers: 8
    pin_memory: true
    max_duration: 16.7
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-05-03 21:57:40 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/jykang/databas

[NeMo I 2024-05-03 21:57:40 features:289] PADDING: 0
[NeMo I 2024-05-03 21:57:41 save_restore_connector:250] Model EncDecCTCModelBPE was successfully restored from /data/jykang/NeMo/nemo_experiments/CTC_Teacher_Decoder_Reusing/Conformer-CTC-BPE-Fitnet-AddLayer/2024-04-04_23-44-32/checkpoints/Conformer-CTC-BPE-Fitnet-AddLayer.nemo.
[NeMo I 2024-05-03 21:57:41 modelPT:1220] Model checkpoint restored from nemo file with path : `/data/jykang/NeMo/nemo_experiments/CTC_Teacher_Decoder_Reusing/Conformer-CTC-BPE-Fitnet-AddLayer/2024-04-04_23-44-32/checkpoints/Conformer-CTC-BPE-Fitnet-AddLayer.nemo`


In [12]:
a.shape

torch.Size([129])

In [13]:
b.shape

torch.Size([129])

In [14]:
c.shape

torch.Size([129])